# Anomaly Detection Models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')
#pd.set_option('max_columns', 200)
from sklearn.model_selection import train_test_split

In [2]:
path = '../data/data_preparation_output.csv'  # ASY local path
# path = "../data/skylab_instagram_datathon_dataset.csv" # gen
df = pd.read_csv(path) #sep='delimiter', header=None) # sep=";", 

In [14]:
df.head()

,period_end_date,compset_group,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,...,timediff_1,diff_1_followers,lag_1_pictures,diff_1_pictures,lag_1_videos,diff_1_videos,lag_1_comments,diff_1_comments,Country Name,Gene Index
0,2017-05-06,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN
1,2017-05-13,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,6.0,3.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-05-20,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,6.0,3.0,...,1.0,NaN,6.0,0.0,3.0,0.0,57.0,0.0,NaN,NaN
3,2017-05-27,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,6.0,3.0,...,1.0,NaN,6.0,0.0,3.0,0.0,57.0,0.0,NaN,NaN
4,2017-06-03,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,24.0,3.0,...,1.0,NaN,6.0,18.0,3.0,0.0,57.0,52.0,NaN,NaN


## Approaches

1. https://scikit-learn.org/stable/modules/outlier_detection.html#novelty-detection

In [3]:
#X_train, X_Test = train_test_split(X, y, test_size=0.33, random_state=42)
from sklearn import svm

In [4]:
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
#clf.fit(df)

ValueError: could not convert string to float: '2017-05-06'

In [5]:
df.columns

Index(['period_end_date', 'compset_group',
       'business_entity_doing_business_as_name', 'legal_entity_name',
       'domicile_country_name', 'ultimate_parent_legal_entity_name',
       'primary_exchange_name', 'followers', 'pictures', 'videos', 'comments',
       'likes', 'compset', 'Year', 'Month', 'Day', 'Weekday',
       'ultimate_parent_vs_legal_entity', 'ultimate_parent_vs_business_entity',
       'legal_entity_vs_business_entity', 'same_ownership',
       'total_involvement', 'total_company_activity', 'conversion_rate_total',
       'return_on_activity', 'ratio_of_videos', 'ratio_of_pictures',
       'likes_per_picture', 'likes_per_video', 'comments_per_picture',
       'comments_per_video',
       'business_entity_doing_business_as_name_cnt_industry',
       'business_entity_doing_business_as_name_cnt_industry_country',
       'followers_sum_industry', 'videos_sum_industry',
       'pictures_sum_industry', 'likes_sum_industry', 'comments_sum_industry',
       'followers_sum_

In [10]:
cols_to_ignore = ['compset', 'period_end_date', 'lag_1_date', 'lag_1_company']
cols_of_interest = ['followers', 'pictures', 'videos', 'comments',
       'likes', 'Year', 'Month', 
       'ultimate_parent_vs_legal_entity', 'ultimate_parent_vs_business_entity',
       'legal_entity_vs_business_entity', 'same_ownership',
       'total_involvement', 'total_company_activity', 'conversion_rate_total',
       'return_on_activity', 'ratio_of_videos', 'ratio_of_pictures',
       'likes_per_picture', 'likes_per_video', 'comments_per_picture',
       'comments_per_video',
       'business_entity_doing_business_as_name_cnt_industry',
       'business_entity_doing_business_as_name_cnt_industry_country',
       'followers_sum_industry', 'videos_sum_industry',
       'pictures_sum_industry', 'likes_sum_industry', 'comments_sum_industry',
       'followers_sum_industry_country', 'videos_sum_industry_country',
       'pictures_sum_industry_country', 'likes_sum_industry_country',
       'comments_sum_industry_country', 'fraction_followers_sum_industry',
       'fraction_followers_sum_industry_country',
       'fraction_videos_sum_industry', 'fraction_videos_sum_industry_country',
       'fraction_pictures_sum_industry',
       'fraction_pictures_sum_industry_country', 'fraction_likes_sum_industry',
       'fraction_likes_sum_industry_country', 'fraction_comments_sum_industry',
       'fraction_comments_sum_industry_country', 'lag_1_followers',
        'timediff_1', 'diff_1_followers',
       'lag_1_pictures', 'diff_1_pictures', 'lag_1_videos', 'diff_1_videos',
       'lag_1_comments', 'diff_1_comments'] 

In [14]:
df_analysis = df[cols_of_interest]
no_nans = df_analysis[~df_analysis.isnull().any(axis=1)]
print(f"{len(df_analysis)} vs {len(no_nans)}")

298040 vs 133998


https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html#sphx-glr-auto-examples-svm-plot-oneclass-py

In [ ]:
clf.fit(no_nans)

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
clf_lof = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination=0.1)
clf_lof.fit(no_nans)